# Week 3 Assignment

Segmenting and Clustering Neighborhoods in Toronto 

## Assignment Section 1 - Data Wrangling

#### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Import Toronto Postal codes  
Source: 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:
!wget -q -O 'PCodes2.csv' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


#### Read Data into Dataframe
Dataframe has 3 columns

In [2]:
# use cleansed csv file
PCodeData ="Wk3PCodes.csv"
df = pd.read_csv(PCodeData)    #df = pd.read_excel(PCodeData)  # use this for Excel file

#rename column 'Postcode' to' PostCode'
df.rename(columns={'Postcode':'PostCode'}, inplace=True)

#verify results
df.head()
#df.shape

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Ignore cells with a Borough that is Not assigned
Clean "Not assigned" values from dataframe 

In [3]:
#remove rows where Borough == 'Not assigned' 
df2 = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df2.head(10)

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Pivot Neighbourhoods into Postcode/Borough rows

In [4]:
# pivot Neighbourhood values on Postcode, Borough index
dft3 = df2.pivot_table(index=['PostCode','Borough'],
                                     #columns='Neighbourhood', 
                                     values='Neighbourhood',
                                     aggfunc=lambda x: ', '.join(x))
# reset index
dft4=dft3.reset_index()
dft4.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Assign 'Not assigned' Neighbourhood with value from Borough 

In [5]:
#locate and replace Neighbourhood value with Borough value where Neighbourhood is Not assigned'
# don't use: dft4.replace("Not assigned", "Queen's Park")
dft4.loc[dft4['Neighbourhood'] == 'Not assigned','Neighbourhood'] =  dft4['Borough']

# verify result
dft4.loc[dft4['PostCode'] == "M7A"]

,PostCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Last Cell uses the .shape method to display # of rows

In [22]:
#print('Number of rows and columns is: ' + dft4.shape)
print('The resulting dataframe has',dft4.shape[0],'rows and',dft4.shape[1],'columns.')

The resulting dataframe has 103 rows and 3 columns.
